# 02 Calibration and Decision Costs

Calibration curves, Brier score, and cost-based thresholds.


## Table of Contents
- [Calibration](#calibration)
- [Brier score](#brier-score)
- [Decision costs](#decision-costs)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Classification notebooks turn the recession label into a **probability model**.
You will learn how to evaluate rare-event prediction and how to choose thresholds intentionally.


## Prerequisites (Quick Self-Check)
- Completed Part 02 (regression basics) or equivalent.
- Comfort interpreting probabilities and trade-offs (false positives vs false negatives).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Reporting only accuracy on imbalanced data.
- Using threshold=0.5 by default without considering costs.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/03_classification/02_calibration_and_costs.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/03_classification/02_calibration_and_costs.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Go beyond accuracy: evaluate probability quality (calibration) and choose thresholds based on decision costs.

Why this matters:
- A recession probability model is only useful if probabilities mean something.
- Threshold selection depends on the cost of false positives vs false negatives.



## Primer: sklearn pipelines (how to avoid preprocessing leakage)

Pipelines prevent a common mistake: fitting preprocessing (scaling, imputation) using information from the test set.

### Why pipelines exist (in one sentence)

> A `Pipeline` ensures that transformations are fit on training data only, then applied to test data.

### The key APIs

- `fit(X, y)`: learn parameters (scaler mean/std, model weights) from training.
- `transform(X)`: apply learned transform to new data.
- `fit_transform(X, y)`: convenience for training data only.

### Minimal pattern (classification)

```python
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
  ("scaler", StandardScaler()),
  ("model", LogisticRegression(max_iter=5000)),
])

# clf.fit(X_train, y_train)
# y_prob = clf.predict_proba(X_test)[:, 1]
```

**Expected output / sanity check**
- you never call `scaler.fit` on the full dataset
- you split by time first, then fit the pipeline on train

### Mini demo: the leakage you’re avoiding (toy)

```python
import numpy as np
from sklearn.preprocessing import StandardScaler

rng = np.random.default_rng(0)
X_train = rng.normal(loc=0.0, scale=1.0, size=(100, 1))
X_test  = rng.normal(loc=2.0, scale=1.0, size=(25, 1))

# WRONG: fit on train+test (leaks the future)
sc_wrong = StandardScaler().fit(np.vstack([X_train, X_test]))
X_test_wrong = sc_wrong.transform(X_test)

# RIGHT: fit on train only
sc_right = StandardScaler().fit(X_train)
X_test_right = sc_right.transform(X_test)

print("test mean after wrong scaling:", float(X_test_wrong.mean()))
print("test mean after right scaling:", float(X_test_right.mean()))
```

### Common pitfalls

- Splitting after preprocessing (leakage).
- Using random splits on time-indexed data (temporal leakage).
- Forgetting `ColumnTransformer` for mixed numeric/categorical columns (if needed).


<a id="calibration"></a>
## Calibration

### Goal
Check whether predicted probabilities match observed frequencies.

A calibrated model:
- among events predicted at 30%, about 30% should occur (in the long run)



### Your Turn (1): Fit a base classifier and get probabilities


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from src.evaluation import time_train_test_split_index

# Load data
path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'target_recession_next_q'
x_cols = ['T10Y2Y_lag1', 'UNRATE_lag1', 'FEDFUNDS_lag1', 'INDPRO_lag1']
df_m = df[[y_col] + x_cols].dropna().copy()

split = time_train_test_split_index(len(df_m), test_size=0.2)
train = df_m.iloc[split.train_slice]
test = df_m.iloc[split.test_slice]

X_train = train[x_cols]
y_train = train[y_col].astype(int)
X_test = test[x_cols]
y_test = test[y_col].astype(int)

clf = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(max_iter=5000)),
])
clf.fit(X_train, y_train)
p_test = clf.predict_proba(X_test)[:, 1]

p_test[:5]



### Your Turn (2): Calibration curve and reliability plot


In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

# TODO: Compute calibration curve
# Hint: calibration_curve(y_true, y_prob, n_bins=...)
prob_true, prob_pred = ...

# TODO: Plot prob_pred vs prob_true with a y=x reference line
...



<a id="brier-score"></a>
## Brier score

### Goal
Compute the Brier score (mean squared error of probabilities).

Lower is better.



### Your Turn: Compute Brier score and interpret


In [ ]:
from sklearn.metrics import brier_score_loss

brier = brier_score_loss(y_test.to_numpy(), p_test)
brier



<a id="decision-costs"></a>
## Decision costs

### Goal
Choose a probability threshold using a simple cost model.

Example framing:
- False negative (miss a recession) might be more costly than a false positive.
- You will encode that as a cost ratio and pick the threshold that minimizes expected cost.



### Your Turn (1): Define costs and compute expected cost across thresholds


In [ ]:
import numpy as np

# Cost of false positives vs false negatives
cost_fp = 1.0
cost_fn = 5.0

thresholds = np.linspace(0.05, 0.95, 19)
costs = []
for thr in thresholds:
    y_pred = (p_test >= thr).astype(int)
    fp = ((y_pred == 1) & (y_test.to_numpy() == 0)).sum()
    fn = ((y_pred == 0) & (y_test.to_numpy() == 1)).sum()
    costs.append(cost_fp * fp + cost_fn * fn)

best_thr = float(thresholds[int(np.argmin(costs))])
best_thr



### Your Turn (2): Plot cost vs threshold


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot thresholds vs costs and mark the best_thr.
...



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Calibration</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_calibration_and_costs — Calibration
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Assume y_te and p from prior notebook
prob_true, prob_pred = calibration_curve(y_te, p, n_bins=6)
plt.plot(prob_pred, prob_true, marker='o')
plt.plot([0,1],[0,1], linestyle='--', color='gray')
plt.xlabel('Predicted')
plt.ylabel('Observed')
plt.title('Calibration curve')
plt.show()
```

</details>

<details><summary>Solution: Brier score</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_calibration_and_costs — Brier score
from sklearn.metrics import brier_score_loss
brier_score_loss(y_te, p)
```

</details>

<details><summary>Solution: Decision costs</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 02_calibration_and_costs — Decision costs
import numpy as np

# Simple cost model: cost_fp and cost_fn
cost_fp = 1.0
cost_fn = 5.0

def expected_cost(thr):
    pred = (p >= thr).astype(int)
    fp = ((pred == 1) & (y_te.to_numpy() == 0)).sum()
    fn = ((pred == 0) & (y_te.to_numpy() == 1)).sum()
    return cost_fp*fp + cost_fn*fn

candidates = np.linspace(0.05, 0.95, 19)
best = min([(thr, expected_cost(thr)) for thr in candidates], key=lambda t: t[1])
best
```

</details>

